In [3]:
import math
!pip install ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


In [4]:
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371.0  # Radius of the Earth in kilometers

    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

denver_airport = ("Denver Airport", (39.8571391027151, -104.67682422525337))
resorts = [
    denver_airport,
    ("Vail",       (39.60634275473704, -106.35502534758554)),
    ("Beaver Creek",(39.601840601122596, -106.53163266107806)),
    ("Breckenridge",(39.48111351412133, -106.07307234912773)),
    ("Keystone",   (39.58186424927065, -105.94362588806374)),
    ("A-Basin",    (39.634289749362566, -105.8714993899124)),
    ("Eldora",     (39.937376571357305, -105.58271078989766)),
    ("Canyons",    (40.685834569379516, -111.55630231684529)),
    ("Heavenly",   (38.9288904430737, -119.90519040528994)),
    ("Northstar",  (39.26481569676928, -120.13316534575092)),
    ("Kirkwood",   (38.68503611762454, -120.06520941879404)),
    ("Afton Alps", (44.85782048122588, -92.78779547430095)),
    ("Mt. Brighton",(42.5410196899345, -83.81160753394613)),
    ("Verbier",    (46.09708737973264, 7.227272162830475)),
    ("Arlberg",    (47.129520437009155, 10.263859008628906)),
    ("The 3 Valleys", (45.34154189826385, 6.586552723873957))
]

# Create the distance matrix
num_locations = len(resorts)
distance_matrix = [
    [
        0 if i == j else int(haversine(resorts[i][1], resorts[j][1]))
        for j in range(num_locations)
    ]
    for i in range(num_locations)
]

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)

start_index = 0  # DIA
best_route    = None
best_distance = float('inf')
best_end      = None

for end_index in range(num_locations):
    if end_index == start_index:
        continue

    # 1) Create manager & model for this (start, end) pair
    manager = pywrapcp.RoutingIndexManager(
        num_locations, 1, start_index, end_index
    )
    routing = pywrapcp.RoutingModel(manager)

    # 2) Register a fresh distance callback
    def make_callback(dist_mat, mgr):
        def distance_callback(from_idx, to_idx):
            from_node = mgr.IndexToNode(from_idx)
            to_node   = mgr.IndexToNode(to_idx)
            return dist_mat[from_node][to_node]
        return distance_callback

    transit_cb = routing.RegisterTransitCallback(
        make_callback(distance_matrix, manager)
    )
    routing.SetArcCostEvaluatorOfAllVehicles(transit_cb)

        # 3) Solve
    solution = routing.SolveWithParameters(search_parameters)
    if solution is None:
        continue

    # 4) Get the route and cost
    index = routing.Start(0)
    route = []
    route_dist = 0
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        route.append(resorts[node][0])
        prev_index = index
        index = solution.Value(routing.NextVar(index))
        route_dist += routing.GetArcCostForVehicle(prev_index, index, 0)

    # append final end node
    route.append(resorts[manager.IndexToNode(index)][0])

# 5) Update best if this is shorter
    if route_dist < best_distance:
        best_distance = route_dist
        best_route    = route
        best_end      = end_index

print(f"Best endpoint: {resorts[best_end][0]}")
print("Open‑TSP optimal route:")
for stop in best_route:
    print("  ", stop)
print(f"Total distance (km): {best_distance}")






TypeError: Wrong number or type of arguments for overloaded function 'new_RoutingIndexManager'.
  Possible C/C++ prototypes are:
    operations_research::RoutingIndexManager::RoutingIndexManager(int,int,operations_research::RoutingIndexManager::NodeIndex)
    operations_research::RoutingIndexManager::RoutingIndexManager(int,int,std::vector< operations_research::RoutingIndexManager::NodeIndex > const &,std::vector< operations_research::RoutingIndexManager::NodeIndex > const &)
